In [1]:
import datasets

glottocode = 'uspa1245'
glosslm_corpus = datasets.load_dataset("lecslab/glosslm-corpus-split")
glottocodes_ID = set(glosslm_corpus['test_ID']['glottocode'])
glottocodes_OOD = set(glosslm_corpus['test_OOD']['glottocode'])

if glottocode in glottocodes_ID:
    id_or_ood = "ID"
elif glottocode in glottocodes_OOD:
    id_or_ood = "OOD"
else:
    raise Exception(
        f"Glottocode should be one of: {list(glottocodes_ID) + list(glottocodes_OOD)}")

# Filter by segmentation and glottocode
glosslm_corpus = glosslm_corpus.filter(
    lambda row: row["is_segmented"] == "no")
train_dataset = glosslm_corpus[f"train_{id_or_ood}"].filter(
    lambda row: row['glottocode'] == glottocode)
eval_dataset = glosslm_corpus[f"test_{id_or_ood}"].filter(
    lambda row: row['glottocode'] == glottocode)

Filter:   0%|          | 0/418718 [00:00<?, ? examples/s]

Filter:   0%|          | 0/104928 [00:00<?, ? examples/s]

Filter:   0%|          | 0/11138 [00:00<?, ? examples/s]

Filter:   0%|          | 0/11940 [00:00<?, ? examples/s]

Filter:   0%|          | 0/7356 [00:00<?, ? examples/s]

Filter:   0%|          | 0/984 [00:00<?, ? examples/s]

Filter:   0%|          | 0/972 [00:00<?, ? examples/s]

Filter:   0%|          | 0/52464 [00:00<?, ? examples/s]

Filter:   0%|          | 0/5970 [00:00<?, ? examples/s]

In [35]:
import time 
import evaluate 



def _compute_chrf(batch):
    scores = []
    for trans in batch['transcription']:
        scores.append(chrf.compute(
            predictions=[trans], references=[["Xpe man anm xkaye' ra l\u00e9k'el rk'i jmaam ."]], word_order=1
        ))
    return {"chrF": scores}

start = time.time()
train_dataset = train_dataset.shuffle()
train_dataset = train_dataset.map(_compute_chrf, batched=True, num_proc=4, keep_in_memory=True) 
train_dataset = train_dataset.sort("chrF", reverse=True) 
time.time() - start 

Map (num_proc=4):   0%|          | 0/9774 [00:00<?, ? examples/s]

{'transcription': ["B'ij chepe' taq xok taq chi iitz' rk'iil .", "ri' li jcholaaj ,", "Toos ri' li pwes tanb'ij iin ,", "xin kwin chij b'itik uchoch .", "iin mkita' chi' chwaaj inb'e ák'l .", "cha' re kompayr rik ,", "Ta' chki' jcholájk ,", "kom ta' eskwela ,", "Ta' chiki' lamaas tye'sájw taq", "kita' raj tijkoj saq'oril ta' raj tichakun .", "No , in ma reech ta' k'ay wichaj li ,", "jun chik jb'iij Xwaan Yaat ,", "jpoomaaj chik jwich jyub' ta'j .", "cha',", "ji' wi' jujun delikad taq ,", "Entoons kla' xuker sik' wunaq ,", "ójr taq qamaam qatiit'.", "cha'.", "t'ijye' jwa ra anm .", "porke jwi'l inrelijiyon .", "loke xqab'an .", "cha'.", "xt'oj b'i man rikra'n iqal juwi'l .", "ajruk' re' xyuuq ,", "Porke maq ra chee' nimi' jq'iij .", "i baa iitz',", "Wákx xak'ayej taq .", '¿Neen chek juun kristyaan li ?.', "pent t'ok chich laj qaxo'l .", "ya xk'isi'n ,", "I jun q'uniik tqat'oj chi riij .", "ta' chiki' oj kwin re .", "ajri'tik ti joob' o waqib' junaab' lajori .", 'juntiir opiisyo wetaam ,

multiprocess.pool.RemoteTraceback: 
"""
Traceback (most recent call last):
  File "/Users/milesper/.pyenv/versions/3.11.6/lib/python3.11/site-packages/multiprocess/pool.py", line 125, in worker
    result = (True, func(*args, **kwds))
                    ^^^^^^^^^^^^^^^^^^^
  File "/Users/milesper/.pyenv/versions/3.11.6/lib/python3.11/site-packages/datasets/utils/py_utils.py", line 623, in _write_generator_to_queue
    for i, result in enumerate(func(**kwargs)):
  File "/Users/milesper/.pyenv/versions/3.11.6/lib/python3.11/site-packages/datasets/arrow_dataset.py", line 3482, in _map_single
    batch = apply_function_on_filtered_inputs(
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/milesper/.pyenv/versions/3.11.6/lib/python3.11/site-packages/datasets/arrow_dataset.py", line 3361, in apply_function_on_filtered_inputs
    processed_inputs = function(*fn_args, *additional_args, **fn_kwargs)
                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/va

In [15]:
len([["Xpe man anm xkaye' ra l\u00e9k'el rk'i jmaam ."]] * 16)

16

In [25]:
chrf = evaluate.load("chrf")